In [2]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from ipynb.fs.full.Funciones_basicas import *
from ipynb.fs.full.Algoritmos import *
from ipynb.fs.full.Visualizaciones import *

In [3]:
from gurobipy import *
import random

In [3]:
def quality_partition(G, solution, Max = True, Norm_2 = True):
    N = len(G.nodes()); M = 1e10; root = 743
    V = [0] * 10 * N
    ordered_solution = find_order(G, V, solution.copy(), root)
    sizes = get_sizes_partition(G, ordered_solution[::-1])

    max_partition = max(sizes) if Max else 0
    norm_2 = sum([i**2 for i in sizes]) if Norm_2 else 0
    
    if not Norm_2:
        return max_partition
    return M * max_partition + norm_2

def get_sizes_partition(G, solution_):
    N = len(G.nodes()); solution = solution_; root = 743
    V = [0] * 10 * N
    sizes = []
    if root not in solution:
        solution.append(root)
    for u in solution:
        sizes.append(get_size(G, V, u))
        V, _ = visit(G, V, u)
    #print(sizes)
    return sizes

def find_order(G, V, sample_, s):
    sample = sample_.copy()
    ans = []; Q = deque([]); Q.append(s)
    while Q:
        u = Q.popleft()
        if u in sample:
            ans.append(u)
            sample.remove(u)
        for v in G.predecessors(u):
            if not V[v]:
                Q.append(v)
    return ans

Instanciamos el grafo

In [4]:
path_geom = '../data/graph_geom_corrected_cycles.csv'
geometry = gpd.read_file(path_geom, GEOM_POSSIBLE_NAMES="geometry", KEEP_GEOM_COLUMNS="NO")
     
path = '../data/corrected_dijkstra.csv'
csv_grafo = pd.read_csv(path, sep=',')

path_pesos = '../data/pesos_tapas.csv'
pesos = pd.read_csv(path_pesos)

out = {1003950, 15004, 15131, 1003744, 15190, 1003746, 13730, 1003167, 13731, 13732, 13733, 14062, 1003206, 16503, 13735, 16094, 16095}

S = set()
for index, row in csv_grafo.iterrows():
    origin = row['self']
    dest = row['other']
    if origin not in out and dest not in out:
        S.add(origin)
        S.add(dest)

S = list(S)
id_ = {}; _id = {}; l = 0
for u in S:
    id_[u] = l; _id[l] = u
    l += 1

Tree = nx.DiGraph()
for index, row in csv_grafo.iterrows():
    origin = row['self']
    dest = row['other']
    if origin not in out and dest not in out:
        Tree.add_edge(id_[origin], id_[dest])
        
G = nx.DiGraph()
for index, row in csv_grafo.iterrows():
    origin = row['self']
    dest = row['other']
    if origin not in out and dest not in out:
        G.add_edge(id_[origin], id_[dest])
    
N = l

W = [0] * N

for u in range(N):
    if pesos[pesos['ID_tapa'] == _id[u]].shape[0] >= 1:  ## tomamos primera columna con el id, si no hay peso = 0
        W[u] = pesos[pesos['ID_tapa'] == _id[u]].iloc[0]['per_predio']

## Funciones útiles para procesar soluciones

In [5]:
def check_solution(V, x, y, check_l = False):
    correct_solution = True
    for i in V:
        if x[i].x == 1:
            for node in V:
                if y[node, i].x == 1 and not node in I[i]:
                    print('Houston tenemos un problema')
                    correct_solution = False
    if check_l:
        if False in [x[l].x <= 1 - y[j,i].x for j in V for i in V for l in Path[j,i] if l != i]:
                   correct_solution = False
    print(f'Solution is correct: {correct_solution} ')
                    
def get_solution(V,x, y, Z, return_y = False):
    print(f'Ideal de mayor tamano: {Z.x}')
    sol = list()
    for i in V:
        if x[i].x == 1:
            sol.append(i)
    if return_y:
        asigments = []
        for i in sol:
            for node in V:
                if y[node, i].x == 1:
                    asigments.append((node, i))
        return sol, asigments
    return sol

## Prueba con diferentes grafos

### Grafo camino

In [41]:
T = nx.path_graph(20, create_using=nx.DiGraph()).reverse()
K = 2
root = 0
V = list(T.nodes())
N = len(V)
#I = {(u, v): u in get_ideal(T, [0] * N, v) and u!=v for u in V for v in V}
I = dict()
P = dict()
for v in V:
    I[v] = get_ideal(T, [0] * 500 * N, v)
    for u in V:
        try:
            P[v,u] = nx.shortest_path(T, source = v, target = u)
        except:
            P[v,u] = list()

### Arbol de instancia con máxima profundidad

In [5]:
T = nx.DiGraph()
max_depth = 18
T = Tree.subgraph([node for node in list(Tree.nodes()) if nx.shortest_path_length(Tree, source = node, target = 744) <= max_depth])
K = 8
root = 744
V = list(T.nodes())
N = len(V)
print(f'{N} nodos en el grafo')

103 nodos en el grafo


In [6]:
I = dict()
Path = dict()
for u in V:
    I[u] = get_ideal(T, [0] * 500 * N, u)
    for v in V:
        if v not in I[u]:
            Path[v,u] = list()
        else:
            Path[v,u] = nx.shortest_path(T, source = v, target = u)

In [12]:
d = dict()
for i in V:
    for j in V:
        if Path[i,j] != list():
            d[i,j] = len(Path[i,j])
        else:
            d[i,j] = N

Instanciación del modelo

In [14]:
model = gurobipy.Model("Muestreo")
x = model.addVars(V, vtype = GRB.BINARY, name ="Nodos raices")
Z = model.addVar(vtype = GRB.CONTINUOUS , obj=1, name ="Ideal de mayor tamano")
y = model.addVars(V, V, vtype = GRB.BINARY, name ="Asignacion de nodo a muestra")


model.addConstr((quicksum(x[i] for i in V) == K), name = "R1")
model.addConstrs((y[j,i] <= x[i] for i in V for j in I[i] ), name = "R2")
model.addConstrs((quicksum(y[i,j] for j in Path[i, root]) == 1 for i in V), name = "R3")
model.addConstrs((x[l] <= 1 - y[j,i] for j in V for i in V for l in Path[j,i] if l != i), name = "R4")
#model.addConstrs((y[i,j] <= x[i]*d[i,j] + (1-x[i]) * N for i in V for j in V))
model.addConstrs((quicksum(y[j,i] for j in I[i]) <= Z for i in V), name = "R5")
model.addConstr((x[root] == 1), name = "Root")


model.setObjective(Z, GRB.MINIMIZE)
model.update()
model.optimize()

Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 14929 rows, 10713 columns and 32929 nonzeros
Model fingerprint: 0xc4ddecf4
Variable types: 1 continuous, 10712 integer (10712 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 8e+00]
Found heuristic solution: objective 78.0000000
Presolve removed 11890 rows and 9074 columns
Presolve time: 0.14s
Presolved: 3039 rows, 1639 columns, 19215 nonzeros
Variable types: 0 continuous, 1639 integer (1638 binary)

Root relaxation: objective 3.140968e+00, 2360 iterations, 0.14 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    3.14097    0  843   78.00000    3.14097  96.0%     -    0s
H    0     0                      72.0000000    3.14097  95.6%     -    0s
H    0     0                      

In [27]:
#model.printAttr("x")
check_solution(V,x,y)
get_solution(V,x,y, Z)

Solution is correct: True 
Ideal de mayor tamano: 24.0


[628, 2163, 744, 2319, 823, 913, 1431, 1433]

## Implementación alternativa

In [56]:
T = nx.path_graph(20, create_using=nx.DiGraph()).reverse()
K = 3
root = 0
V = list(T.nodes())
N = len(V)
#I = {(u, v): u in get_ideal(T, [0] * N, v) and u!=v for u in V for v in V}
I = dict()
P = dict()
Path = dict()
for u in V:
    I[u] = get_ideal(T, [0] * 500 * N, u)
    for v in V:
        if v not in I[u]:
            P[v,u] = N**10
            Path[v,u] = list()
        else:
            Path[v,u] = nx.shortest_path(T, source = v, target = u)
            P[v,u] = len(Path[v,u]) - 1 

In [6]:
T = nx.DiGraph()
max_depth = 20
T = Tree.subgraph([node for node in list(Tree.nodes()) if nx.shortest_path_length(Tree, source = node, target = 744) <= max_depth])
K = 8
root = 744
V = list(T.nodes())
N = len(V)
print(f'{N} nodos en el grafo')


176 nodos en el grafo


In [7]:
I = dict()
P = dict()
Path = dict()
eps = 0.01
for u in V:
    I[u] = get_ideal(T, [0] * 500 * N, u)
    for v in V:
        if v not in I[u]:
            P[v,u] = N**10
            Path[v,u] = list()
        else:
            Path[v,u] = nx.shortest_path(T, source = v, target = u)
            P[v,u] = len(Path[v,u]) + eps

In [166]:
model = gurobipy.Model("Muestreo")
x = model.addVars(V, vtype = GRB.BINARY, name ="Nodos raices")
Z = model.addVar(vtype = GRB.CONTINUOUS, obj=1,  name ="Ideal de mayor tamano")
y = model.addVars(V, V, vtype = GRB.BINARY, name ="Asignacion de nodo a muestra")

model.addConstr((quicksum(x[i] for i in V) == K), name = "R1")
model.addConstrs((y[j,i] <= x[i] for i in V for j in I[i] ), name = "R2")
model.addConstrs((quicksum(y[i,j] for j in Path[i, root]) == 1 for i in V), name = "R3")
model.addConstrs((quicksum(y[j,i] for j in I[i]) <= Z for i in V), name = "R5")
model.addConstr((x[root] == 1), name = "Root")

alpha = 0.3
M = alpha * quicksum(quicksum(P[i,j]*y[i,j] for j in V) for i in V ) + Z
model.setObjective(M, GRB.MINIMIZE)
model.update()
model.optimize()

Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 3488 rows, 31153 columns and 12889 nonzeros
Model fingerprint: 0xe97d1352
Variable types: 1 continuous, 31152 integer (31152 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e-01, 9e+21]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 8e+00]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 238 rows and 27845 columns
Presolve time: 0.10s
Presolved: 3250 rows, 3308 columns, 12413 nonzeros
Variable types: 0 continuous, 3308 integer (3307 binary)
Found heuristic solution: objective 754.5280000
Found heuristic solution: objective 405.3280000

Root relaxation: objective 2.548561e+02, 2045 iterations, 0.13 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  254.85605    0 

In [167]:
check_solution(V, x, y, check_l = True)
solution_alternative, asigments = get_solution(V,x, y, Z, True)

Solution is correct: False 
Ideal de mayor tamano: 34.0


## warm start para la formulación original?

In [120]:
model = gurobipy.Model("Muestreo")
x = model.addVars(V, vtype = GRB.BINARY, name ="Nodos raices")
Z = model.addVar(vtype = GRB.CONTINUOUS , obj=1, name ="Ideal de mayor tamano")
y = model.addVars(V, V, vtype = GRB.BINARY, name ="Asignacion de nodo a muestra")
        
model.addConstr((quicksum(x[i] for i in V) == K), name = "R1")
model.addConstrs((y[j,i] <= x[i] for i in V for j in I[i] ), name = "R2")
model.addConstrs((quicksum(y[i,j] for j in Path[i, root]) == 1 for i in V), name = "R3")
model.addConstrs((x[l] <= 1 - y[j,i] for j in V for i in V for l in Path[j,i] if l != i), name = "R4")
model.addConstrs((quicksum(y[j,i] for j in I[i]) <= Z for i in V), name = "R5")
model.addConstr((x[root] == 1), name = "Root")

model.setObjective(Z, GRB.MINIMIZE)

solution_alternative = [410, 665, 671, 676, 680, 2149, 744, 2167]
for root in solution_alternative:
    x[root].start = 1.0
# for pair in asigments:
#     y[pair].start = 1.0
#Z.start = 24.0

model.update()
model.optimize()

Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 3966 rows, 1807 columns and 8887 nonzeros
Model fingerprint: 0xb0f3a641
Variable types: 1 continuous, 1806 integer (1806 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 8e+00]

User MIP start produced solution with objective 26 (0.01s)
Loaded user MIP start with objective 26

Presolve removed 3021 rows and 1287 columns
Presolve time: 0.17s
Presolved: 945 rows, 520 columns, 5274 nonzeros
Variable types: 0 continuous, 520 integer (519 binary)

Root relaxation: objective 1.901959e+00, 502 iterations, 0.04 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    1.90196    0  257   26.00000    1.90196  92.7%     -    0s
H    0     0                      15.0000000    1.90196  87.3%     

In [152]:
get_solution(V, x, y, Z)

Ideal de mayor tamano: 24.0


[628, 2163, 744, 2319, 1431, 1433, 422, 963]

## Relajación lagrangiana

In [155]:
T = nx.DiGraph()
max_depth = 20
T = Tree.subgraph([node for node in list(Tree.nodes()) if nx.shortest_path_length(Tree, source = node, target = 744) <= max_depth])
K = 8
root = 744
V = list(T.nodes())
N = len(V)
print(f'{N} nodos en el grafo')

176 nodos en el grafo


In [156]:
I = dict()
Path = dict()
for u in V:
    I[u] = get_ideal(T, [0] * 500 * N, u)
    for v in V:
        if v not in I[u]:
            Path[v,u] = list()
        else:
            Path[v,u] = nx.shortest_path(T, source = v, target = u)

Modelo original para comparar resultados

In [157]:
model = gurobipy.Model("Muestreo")
x = model.addVars(V, vtype = GRB.BINARY, name ="Nodos raices")
Z = model.addVar(vtype = GRB.CONTINUOUS, lb = math.ceil(N/K), obj=1, name ="Ideal de mayor tamano")
y = model.addVars(V, V, vtype = GRB.BINARY, name ="Asignacion de nodo a muestra")
        
model.addConstr((quicksum(x[i] for i in V) == K), name = "R1")
model.addConstrs((y[j,i] <= x[i] for i in V for j in I[i] ), name = "R2")
model.addConstrs((quicksum(y[i,j] for j in Path[i, root]) == 1 for i in V), name = "R3")
model.addConstrs((x[l] <= 1 - y[j,i] for j in V for i in V for l in Path[j,i] if l != i), name = "R4")
model.addConstrs((quicksum(y[j,i] for j in I[i]) <= Z for i in V), name = "R5")
model.addConstr((x[root] == 1), name = "Root")

model.setObjective(Z, GRB.MINIMIZE)
model.update()
model.optimize()

Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 31119 rows, 31153 columns and 68151 nonzeros
Model fingerprint: 0xb08e1c29
Variable types: 1 continuous, 31152 integer (31152 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 2e+01]
  RHS range        [1e+00, 8e+00]
Presolve removed 25352 rows and 28020 columns
Presolve time: 0.56s
Presolved: 5767 rows, 3133 columns, 38882 nonzeros
Variable types: 0 continuous, 3133 integer (3132 binary)
Found heuristic solution: objective 76.0000000

Root relaxation: objective 2.200000e+01, 2501 iterations, 0.52 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   22.00000    0  399   76.00000   22.00000  71.1%     -    3s
H    0     0                      68.0000000   22.00000  67.6%     -    3s
H    0     0                     

In [158]:
check_solution(V, x, y, check_l = True)
solution_alternative, asigments = get_solution(V,x, y, Z, True)
solution_alternative

Solution is correct: True 
Ideal de mayor tamano: 24.0


[628, 2163, 744, 2319, 823, 1431, 1433, 963]

Relajación lagrangeana

In [188]:
import math
model = gurobipy.Model("Relajacion")
x = model.addVars(V, vtype = GRB.BINARY, name ="Nodos raices")
Z = model.addVar(vtype = GRB.CONTINUOUS , obj=1, lb = math.ceil(N/K),  name ="Ideal de mayor tamano")
y = model.addVars(V, V, vtype = GRB.CONTINUOUS,lb = 0, name ="Asignacion de nodo a muestra")
        
#model.addConstr((quicksum(x[i] for i in V) == K), name = "R1") # No va
model.addConstrs((y[j,i] <= x[i] for i in V for j in I[i] ), name = "R2") #  va
#model.addConstrs((quicksum(y[i,j] for j in Path[i, root]) == 1 for i in V), name = "R3") # no va
#model.addConstrs((x[l] <= 1 - y[j,i] for j in V for i in V for l in Path[j,i] if l != i), name = "R4")
#model.addConstrs((quicksum(y[j,i] for j in I[i]) <= Z for i in V), name = "R5")
#model.addConstr((x[root] == 1), name = "Root") # no va


penalties_0 = [model.addVar(vtype = GRB.CONTINUOUS) for j in V for i in V for l in Path[j,i] if l != i]
penalties_1 = [model.addVar(vtype = GRB.CONTINUOUS) for i in V]
penalties_2 = [model.addVar(vtype = GRB.CONTINUOUS) for i in V for j in Path[i, root] ]


# Crear un conjunto que contenga: j in V for i in V for l in Path[j,i] if l != i
# Dualized constraints: sum j: x_ij <= 1 for all i
model.update()
pi = 0
for j in V:
    for i in V:
        for l in Path[j,i]:
            if l != i:
                #(x[l] <= 1 - y[j,i]) 
                p = penalties_0[pi]
                model.addConstr( p >= y[j,i] + x[l] - 1 )
                pi += 1
                
pi = 0
for i in V:
    p = penalties_1[pi]
    model.addConstr( p >= quicksum(y[j,i] for j in I[i]) - Z )
    pi += 1

pi = 0                
for i in V:
    for j in Path[i, root]:
        p = penalties_2[pi]
        #model.addConstr( p >= quicksum(y[j,i] for j in I[i]) - Z )
        model.addConstr( p == quicksum(y[i,j] for j in Path[i, root]) - 1 ) 
        pi += 1



# u[i] = Lagrangian Multiplier for the set packing contraint i
u_0 = [ 1 for j in V for i in V for l in Path[j,i] if l != i]
u_1 = [ 1 for i in V]
u_2 = [ 1 for i in V for j in Path[i, root] ]
u_k = 5
u_r = 1

#model.setParam('OutputFlag', False)
# +  quicksum(u * p_j for u, p_j in zip(u_0, penalties_0) )
model.setObjective( Z + quicksum(u * p_j for u, p_j in zip(u_0, penalties_0) )
                   + quicksum(u * p_j for u, p_j in zip(u_2, penalties_2) )
                                 +  quicksum(u * p_j for u, p_j in zip(u_1, penalties_1))
                   +  u_k * (quicksum(x[i] for i in V) - K) + u_r * (1 - x[root]) , GRB.MINIMIZE)
model.update()
model.optimize()

Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 34075 rows, 62094 columns and 154177 nonzeros
Model fingerprint: 0xcd303072
Variable types: 61918 continuous, 176 integer (176 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 2e+01]
  Bounds range     [1e+00, 2e+01]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 1073.0000000
Presolve removed 3314 rows and 30991 columns
Presolve time: 0.69s
Presolved: 30761 rows, 31103 columns, 92102 nonzeros
Variable types: 30928 continuous, 175 integer (175 binary)

Root relaxation: objective -7.000000e+00, 2118 iterations, 0.21 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0      -7.0000000   -7.00000  0.00%     -    1s

Explored 0 nodes (2118 simplex iterations) in 1.09 seconds
Thread count was 4 (of 4 available proces

In [189]:
check_solution(V, x, y, check_l = True)
solution_alternative, asigments = get_solution(V,x, y, Z, True)
solution_alternative

Solution is correct: True 
Ideal de mayor tamano: 176.0


[744]

In [105]:
model = gurobipy.Model("Relajacion")
x = model.addVars(V, vtype = GRB.BINARY, name ="Nodos raices")
Z = model.addVar(vtype = GRB.CONTINUOUS , obj=1, name ="Ideal de mayor tamano")
y = model.addVars(V, V, vtype = GRB.BINARY, name ="Asignacion de nodo a muestra")
        
model.addConstr((quicksum(x[i] for i in V) == K), name = "R1")
model.addConstrs((y[j,i] <= x[i] for i in V for j in I[i] ), name = "R2")
model.addConstrs((quicksum(y[i,j] for j in Path[i, root]) == 1 for i in V), name = "R3")
#model.addConstrs((x[l] <= 1 - y[j,i] for j in V for i in V for l in Path[j,i] if l != i), name = "R4")
model.addConstrs((quicksum(y[j,i] for j in I[i]) <= Z for i in V), name = "R5")
model.addConstr((x[root] == 1), name = "Root")


penalties = [model.addVar(vtype = GRB.BINARY) for j in V for i in V for l in Path[j,i] if l != i]


# Crear un conjunto que contenga: j in V for i in V for l in Path[j,i] if l != i
# Dualized constraints: sum j: x_ij <= 1 for all i
model.update()
pi = 0
for j in V:
    for i in V:
        for l in Path[j,i]:
            if l != i:
                #(x[l] <= 1 - y[j,i]) 
                p = penalties[pi]
                model.addConstr( p >= y[j,i] + x[l] - 1 )
                pi += 1



# u[i] = Lagrangian Multiplier for the set packing contraint i
u = [2 for j in V for i in V for l in Path[j,i] if l != i]
eps = 10e-6
#model.setParam('OutputFlag', False)


for k in range(1, 101):
    
    model.setObjective(Z + quicksum(u * p_j for u, p_j in zip(u, penalties) ), GRB.MINIMIZE)
    model.update()
    model.optimize()
    print(f'iteration {k}, obj = {model.objVal}')
    
    stop = True
    for u_i, p_i in zip(u, penalties):
        if abs(u_i) > eps and abs(p_i.x) > eps:
            stop = False
            break

    if stop:
        print('primal feasible & optimal')
        break

    else:
        s = 1.0 / k
        for p in range(len(penalties)):
            u[i] = max(u[i] - s*(penalties[p].x), 0.0)

print('objective =', model.objVal)


Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 40299 rows, 81104 columns and 123970 nonzeros
Model fingerprint: 0xe601a0a4
Variable types: 1 continuous, 81103 integer (81103 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 2e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 8e+00]
Presolve removed 285 rows and 41021 columns
Presolve time: 1.69s
Presolved: 40014 rows, 40083 columns, 123400 nonzeros
Variable types: 0 continuous, 40083 integer (40082 binary)

Explored 0 nodes (0 simplex iterations) in 2.22 seconds
Thread count was 4 (of 4 available processors)

Solution count 0

Solve interrupted
Best objective -, best bound 1.000000000000e+00, gap -
iteration 1, obj = inf


AttributeError: Unable to retrieve attribute 'x'

In [95]:
check_solution(V, x, y, check_l = True)
solution_alternative, asigments = get_solution(V,x, y, Z, True)
solution_alternative

Solution is correct: False 
Ideal de mayor tamano: 24.0


[628, 2163, 744, 2319, 904, 913, 1434, 499]

In [29]:
# for p in penalties:
#     if p.x  < 1:
#         print('x')
#sum([p.x for p in penalties ])
model.computeIIS()
model.write("model.ilp")


IIS computed: 17 constraints, 12 bounds
IIS runtime: 0.00 seconds
